In [1]:
import pandas as pd

def load_data(file_path, chunksize=100000):
    """Load the data into a DataFrame in chunks."""
    all_data = pd.DataFrame()
    for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
        all_data = pd.concat([all_data, chunk])
    return all_data

# Business Dataset

In [2]:
business = pd.read_json('datasets/yelp_academic_dataset_business.json', lines=True)
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
# Filter for businesses in Tucson
tucson_buzz = business[business.city.str.contains('Tucson', regex=True) == True].copy(deep=True)
tucson_buzz.reset_index(drop=True, inplace=True)

# Fix naming convention
tucson_buzz['city'] = 'Tucson'

# Filter for restaurants
tucson_restaurants = tucson_buzz[tucson_buzz.categories.str.contains('Restaurants|Restaurant', regex=True) == True].copy(deep=True)


In [21]:
# Filter for restaurants with reviews
tucson = tucson_restaurants[tucson_restaurants.review_count >= 100].copy(deep=True)
tucson.reset_index(drop=True, inplace=True)
tucson.to_csv('data/tucson_business.csv', index=False)

# Reviews Dataset

In [24]:
reviews = load_data('datasets/yelp_academic_dataset_review.json')

In [25]:
reviews_df = reviews[reviews['business_id'].isin(tucson['business_id'])].copy(deep=True)
print(f"reviews shape: {reviews_df.shape}")

reviews shape: (191113, 9)


In [26]:
reviews_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
14,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4,0,2,0,The bun makes the Sonoran Dog. It's like a snu...,2011-10-27 17:12:05
75,ymhbOMW63B_vGaRFR3XT0A,yZdAhQ_KmKuCZmbBy9YDQg,5Ce3lZksYVkCbrihqylVHQ,5,0,0,0,I just started going to Sushi Nara this month ...,2014-07-25 17:56:26
147,ypFqmURIY41F4pWaMW1VrQ,hToW2eoTAYlOcuNz7I73Cg,7L1kXfwU5XM6f6Jxo_L7Yg,5,1,0,0,We've been coming here since I was a kid. The ...,2015-07-03 22:36:04
185,M7h8K7PrLrN8yQb3RhHLdA,CysTz612pTix75Fye6-D4Q,WLiqfxv_GhFFA5sm878a2w,5,0,1,1,Blew $80 on food for the family was worth it f...,2013-12-29 03:15:34
194,_u331rYIp7qJ7aWvLup9Ug,BUpAhzsbbKbTqyBjT8YnNQ,wa_bwyY57etHjtJ2Fw0E3g,5,0,0,0,Called in a Mediterranean chicken shawarma. Go...,2014-04-01 05:00:09


In [27]:
reviews_df.reset_index(drop=True, inplace=True)
reviews_df.to_csv('data/tucson_reviews.csv', index=False)

# Users Dataset

In [22]:
users = load_data('datasets/yelp_academic_dataset_user.json')

In [23]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [28]:
users_df = users[users['user_id'].isin(reviews_df['user_id'])].copy(deep=True)
print(f"users shape: {users_df.shape}")

users shape: (68783, 22)


In [29]:
users_df.reset_index(drop=True, inplace=True)
users_df.to_csv('data/tucson_users.csv', index=False)